In [2]:
import pandas as pd
import os

In [ ]:
# Avergae GCP per district
# Number of at_risks students per district
# Engagement rate per district


# Extract District Information 
# 001 Washington District
# 002 New York District

# Create a table for districts
#distric_id
# district_name

In [64]:
# Define file paths
raw_data = '../data/raw/'
processed_data = '../data/processed/'

In [ ]:
def create_district_mapping(filepath: str) -> str:
    filename = filepath.split('/')[-1].split(".")[0]
    
    district_mapping = {
        'school_alpha': 'Washington District',
        'school_beta': 'New York District',
        'school_gamma': 'New York District'
    }
    return district_mapping[filename]

In [65]:
def create_district_csv(filepath: str,data:pd.read_csv) -> dict:
    filename = filepath.split('/')[-1].split(".")[0]
    
    district_mapping = {
        'school_alpha': '001_Washington_District',
        'school_beta': '002_New_York_District',
        'school_gamma': '002_New_York_District'
    }

    data.to_csv(os.path.join(processed_data,f"{district_mapping[filename]}.csv"), index=False)
    

In [60]:
create_district_mapping("school_alpha.csv")

'Washington District'

In [58]:
def clean_data(file_path):

    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    
    column_map = {
    'id': 'student_id',
    'full_name': 'full_name',
    'sex': 'gender',
    'maths': 'math_score',
    'english_language': 'english_score',
    'sci': 'science_score',
    'attendance_%': 'attendance_rate',
    'mathscore': 'math_score',
    'englishscore': 'english_score',
    'english_score': 'english_score',
    'science_score': 'science_score',
    'attendance': 'attendance_rate',
    'Sex': 'gender',
    'Maths': 'math_score'
}

    df.rename(columns=column_map, inplace=True)


    df['math_score'] = pd.to_numeric(df['math_score'], errors='coerce')
    df['english_score'] = pd.to_numeric(df['english_score'], errors='coerce')
    df['science_score'] = pd.to_numeric(df['science_score'], errors='coerce')
    df['attendance_rate'] = pd.to_numeric(df['attendance_rate'], errors='coerce')

    return df[['student_id', 'full_name', 'gender', 'math_score',
                'english_score', 'science_score', 'attendance_rate', 'term']]

In [61]:
file_names = ['school_alpha.csv', 'school_beta.csv', 'school_gamma.csv']
raw_files = [os.path.join(raw_data, f) for f in file_names]

# raw_files

In [66]:
more_data = []
for file in raw_files:
    data = clean_data(file)
    data['district'] = create_district_mapping(file)
    more_data.append(data)
all_data = pd.concat(more_data, ignore_index=True)
 

In [67]:
for file in raw_files:
    data = clean_data(file)
    data['district'] = create_district_mapping(file)
    create_district_csv(file,data)

In [68]:
all_data.to_csv(os.path.join(processed_data, 'all_std_districts.csv'), index=False)